In [6]:
import httpx
import pandas as pd
from IPython.display import display, HTML
from ipywidgets import widgets, VBox


def fetch_recipes_from_api(ingredients):
    """Fetch recipes using the Spoonacular API."""
    api_key = "ebf8b29918e54003a23c1573e5bf368f"  
    url = f"https://api.spoonacular.com/recipes/findByIngredients"
    
    params = {
        "ingredients": ",".join(ingredients),
        "number": 30, 
        "apiKey": api_key
    }
    
    response = httpx.get(url, params=params)
    if response.status_code != 200:
        print(f"Failed to fetch recipes. HTTP Status Code: {response.status_code}")
        return None
    
    recipes = response.json()
    recipe_list = []
    for recipe in recipes:
        recipe_list.append({
            "Title": recipe['title'],
            "Link": f"https://spoonacular.com/recipes/{recipe['title'].replace(' ', '-')}-{recipe['id']}"
        })
    
    return pd.DataFrame(recipe_list)


ingredient_input = widgets.Text(
    description='Ingredients:',
    placeholder='e.g., chicken, rice, tomato',
    layout=widgets.Layout(width='100%')
)
output = widgets.Output()

def handle_input(change):
    """Process user input and display recipes."""
    user_input = change['new'].strip()
    if not user_input:
        return 
    
    with output:
        output.clear_output()  
        ingredients = [ing.strip() for ing in user_input.split(',')]
        recipes_df = fetch_recipes_from_api(ingredients)
        
        if recipes_df is None or recipes_df.empty:
            display(HTML("<p style='color:red;'>No recipes found. Please try different ingredients.</p>"))
        else:
            display(HTML(f"<h3>Recipes for: <em>{', '.join(ingredients)}</em></h3>"))
            display(HTML(recipes_df.to_html(render_links=True, escape=False, classes='table table-bordered', index=False)))


ingredient_input.observe(handle_input, names='value')


display(HTML("<h1>Recipe Finder</h1>"))
display(VBox([ingredient_input, output]))

